In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import my_util,my_scaler,read_file,arima
import pyflux as pf
from sklearn.metrics import mean_squared_error

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 18,9

/home/zhiyedan/anaconda2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
data = read_file.read_aws('art_daily_small_noise.csv')
data = data.to_frame()

diffn,_ = arima.best_diff(data,col='value')
data,diffn_col_name,bck_df = arima.add_diff_col(data,'value',diffn)

train,test = my_util.split_train_test(data,0.7)

order = arima.best_pq(train[diffn_col_name])
aic_order = order.aic_min_order
ar,ma = aic_order[0],aic_order[1]
print ar,ma

In [4]:
model = pf.ARIMA(data=train, ar=ar, ma=ma, target=diffn_col_name, family=pf.Normal()).fit('MLE')
# x = model.fit("MLE")
# x.summary()

In [7]:
model.predict()

AttributeError: 'MLEResults' object has no attribute 'predict'

In [6]:
train[-10:-1]
test[:10]

,value,diff0
timestamp,,
2014-04-10 18:20:00,34.620433,34.620433
2014-04-10 18:25:00,30.039187,30.039187
2014-04-10 18:30:00,31.464218,31.464218
2014-04-10 18:35:00,31.672643,31.672643
2014-04-10 18:40:00,32.633664,32.633664
2014-04-10 18:45:00,29.259101,29.259101
2014-04-10 18:50:00,30.251249,30.251249
2014-04-10 18:55:00,34.761532,34.761532
2014-04-10 19:00:00,24.229676,24.229676


,value,diff0
timestamp,,
2014-04-10 19:10:00,20.246916,20.246916
2014-04-10 19:15:00,21.624989,21.624989
2014-04-10 19:20:00,22.973254,22.973254
2014-04-10 19:25:00,21.392851,21.392851
2014-04-10 19:30:00,24.234947,24.234947
2014-04-10 19:35:00,22.999824,22.999824
2014-04-10 19:40:00,20.482141,20.482141
2014-04-10 19:45:00,24.232344,24.232344
2014-04-10 19:50:00,24.450793,24.450793


In [7]:
# data['recovery'] = arima.recovery_diff(data[diffn_col_name],diffn,bck_df['value'][0])